In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import sklearn
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from preprocessor import MNIST_Dataset
from preprocessor import compute_accuracy
from preprocessor import train
from preprocessor import train2

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [2]:
batch_size = 64
mnist_real_train = MNIST_Dataset("MNIST/processed/training.pt")

mnist_train, mnist_validation = data.random_split(mnist_real_train, (48000, 12000))


train_dataloader = DataLoader(mnist_train, batch_size=batch_size)
validation_dataloader = DataLoader(mnist_validation, batch_size=batch_size)

mnist_test = MNIST_Dataset("MNIST/processed/test.pt")
test_dataloader = DataLoader(mnist_test, batch_size=batch_size)

len(mnist_real_train), len(mnist_train), len(mnist_validation), len(mnist_test)

(60000, 48000, 12000, 10000)

Classifier 1 

In [4]:
from Classifier_1 import SimpleFeedForwardNetwork
modelC1 = SimpleFeedForwardNetwork().to(device)
print(modelC1)
opt = optim.Adam(modelC1.parameters())
cost = torch.nn.CrossEntropyLoss()
train(mnist_train, mnist_validation,modelC1,cost,opt,n_epoch = 50)


SimpleFeedForwardNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Tanh()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Tanh()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): Tanh()
  )
)


In [19]:
#this functionality was inspiried by the example in pytorch documentation https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
corr = 0
tot = 0 
with torch.no_grad(): 
    for data in test_dataloader: 
        images, labels = data
        outputs = modelC1(images)
        _, predicted = torch.max(outputs.data, 1)
        tot += labels.size(0)
        corr += (predicted == labels).sum().item()
print(corr, "out of", tot, "correct")    
print((corr/tot)*100,'%')

9762 out of 10000 correct
97.61999999999999 %


Classifier 2 

In [32]:
from Classifier_2 import FeedForwardNetwork
modelC2 = FeedForwardNetwork().to(device)
print(modelC2)
opt = optim.Adam(modelC2.parameters())
cost = torch.nn.CrossEntropyLoss()
train(mnist_train, mnist_validation,modelC2,cost,opt,n_epoch = 100)

FeedForwardNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [33]:
corr = 0
tot = 0 
with torch.no_grad(): 
    for data in test_dataloader: 
        images, labels = data
        outputs = modelC2(images)
        _, predicted = torch.max(outputs.data, 1)
        tot += labels.size(0)
        corr += (predicted == labels).sum().item()
print(corr, "out of", tot, "correct")    
print((corr/tot)*100,'%')

6842 out of 10000 correct
68.42 %


Classifier 3 

In [34]:
from Classifier_3 import MultiLayerFeedForwardNetwork
modelC3 = MultiLayerFeedForwardNetwork().to(device)
print(modelC3)
opt = optim.SGD(modelC3.parameters(), lr=0.1, momentum=0.9)
cost = torch.nn.CrossEntropyLoss()
train(mnist_train, mnist_validation,modelC3,cost,opt,n_epoch = 100)

MultiLayerFeedForwardNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): Sigmoid()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): Sigmoid()
    (8): Linear(in_features=512, out_features=10, bias=True)
    (9): Softmax(dim=1)
  )
)


In [35]:
corr = 0
tot = 0 
with torch.no_grad(): 
    for data in test_dataloader: 
        images, labels = data
        outputs = modelC2(images)
        _, predicted = torch.max(outputs.data, 1)
        tot += labels.size(0)
        corr += (predicted == labels).sum().item()
print(corr, "out of", tot, "correct")    
print((corr/tot)*100,'%')

6842 out of 10000 correct
68.42 %


Classifier 4

In [27]:
from Classifier_4 import ConvNeuralNetwork
modelC4 = ConvNeuralNetwork().to(device)
print(modelC4)
opt = optim.Adam(modelC4.parameters())
cost = torch.nn.CrossEntropyLoss()
train2(mnist_train, mnist_validation,modelC4,cost,opt,n_epoch=100)

ConvNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
    (3): Flatten(start_dim=1, end_dim=-1)
    (4): Linear(in_features=500, out_features=10, bias=True)
  )
)


In [28]:
corr = 0
tot = 0 
with torch.no_grad(): 
    for data in test_dataloader: 
        images, labels = data
        outputs = modelC4(images)
        _, predicted = torch.max(outputs.data, 1)
        tot += labels.size(0)
        corr += (predicted == labels).sum().item()
print(corr, "out of", tot, "correct")    
print((corr/tot)*100,'%')

9732 out of 10000 correct
97.32 %


Classifier 5

In [29]:
from Classifier_5 import CNNDropNet
modelC5 = CNNDropNet().to(device)
print(modelC5)
opt = optim.Adam(modelC5.parameters())
cost = torch.nn.CrossEntropyLoss()
train2(mnist_train, mnist_validation,modelC5,cost,opt,n_epoch=100)

CNNDropNet(
  (layers): Sequential(
    (0): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
    (3): Flatten(start_dim=1, end_dim=-1)
    (4): Dropout(p=0.2, inplace=False)
    (5): Linear(in_features=500, out_features=10, bias=True)
  )
)


In [23]:
corr = 0
tot = 0 
with torch.no_grad(): 
    for data in test_dataloader: 
        images, labels = data
        outputs = modelC5(images)
        _, predicted = torch.max(outputs.data, 1)
        tot += labels.size(0)
        corr += (predicted == labels).sum().item()
print(corr, "out of", tot, "correct")    
print((corr/tot)*100,'%')

9774 out of 10000 correct
97.74000000000001 %


K-Fold testing implementation 

In [9]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, ConcatDataset

IndentationError: expected an indented block (2081724090.py, line 129)